In [1]:
# Reads the COM of all Carbon nanotubes and writes to a file the number of clusters as well as the size of the biggest one

In [2]:
#-----------------#
# IMPORT PACKAGES #
#-----------------#


import sys
import argparse
import numpy as np
import scipy as sp
import MDAnalysis as mda
import time



#----------------------#
# FUNCTION DEFINITIONS #
#----------------------#


def distance(x0, x1, dimensions):
    """Calculate the distance concerning periodic boundary conditions."""
    
    delta = np.abs(x0 - x1)
    delta = np.where(delta > 0.5 * dimensions, dimensions - delta, delta)
    
    return np.sqrt((delta ** 2).sum(axis=-1))


def connection_list(coord, dim, cutoff):
    """Create a connection matrix for the coordinates based on the cutoff radius"""
    
    n = len(coord)
    
    connections = []
    
    for i in xrange(n): 
        newconnect = []        
        for j in xrange(n):
            dist = distance(coord[i,:3],coord[j,:3],dim[:3])
            if dist <= cutoff:
                newconnect.append(j)
        connections.append(newconnect)
    
    return connections
    

def clustering(coord, dim, cutoff):
    """Calculate the number of clusters and the size of the biggest cluster."""
    
    # Create the connection list
    connections          = connection_list(coord,dim,cutoff)

    # Initialize the counting variables
    n                    = len(coord)
    number_of_clusters   = 0
    biggest_cluster_size = 0 
    cluster_index        = np.zeros(n, dtype=np.int)
    cluster_size         = []
    
    # Count the clusters and number of elements per cluster
    for i in xrange(n): 
        if cluster_index[i] == 0:
            number_of_clusters += 1
            cluster_index[i] = number_of_clusters
            cluster_size.append(1)
            queue = connections[i]
            while len(queue) > 0:
                newqueue = []
                for j in queue:
                    if cluster_index[j] == 0:
                        cluster_index[j] = number_of_clusters
                        cluster_size[cluster_index[i]-1] += 1
                        for k in connections[j]:
                            newqueue.append(k)
                queue = newqueue         
                
    biggest_cluster_size = np.max(cluster_size)
            
    return number_of_clusters, biggest_cluster_size


In [3]:
#----------------#
#  MAIN ROUTINE  #
#----------------#


start = time.time()


# PARSE THE ARGUMENTS

parser = argparse.ArgumentParser(description='Reads the COM of all Carbon nanotubes and writes to a file the number of clusters as well as the size of the biggest one.')
parser.add_argument( '-c', '--cutoff',  type=float, default=22.5,             help='cut-off radius for clustering analysis' )
parser.add_argument( '-b', '--begin',   type=int,   default=0,                help='frame to begin with'                    )
parser.add_argument( '-e', '--end',     type=int,   default=None,             help='last frame'                             )
parser.add_argument( '-s', '--skip',    type=int,   default=1,                help='use only every sth frame'               )
parser.add_argument( '-t', '--topol',   type=str,   default='martini_md.tpr', help='topology file'                          )
parser.add_argument( '-x', '--traj',    type=str,   default='martini_md.xtc', help='trajectory file'                        )
parser.add_argument( '-o', '--outfile', type=str,   default='clustering.dat', help='output file'                            ) 
testargs = '-s 1 -t ../../POPC-100CNT-12-8-f00-SNda/simulation/martini_md.tpr -x ../../POPC-100CNT-12-8-f00-SNda/simulation/whole.xtc'
args     = parser.parse_args(testargs.split())
#args    = parser.parse_args()


# LOAD NECESSARY DATA

universe     = mda.Universe(args.topol,args.traj)
cnt_atoms    = universe.select_atoms("resname CNT")
cnt_residues = cnt_atoms.residues


# GO THROUGH THE TRAJECTORY AND DO ANALYSIS

with open(args.outfile, 'w') as output:

    # If no end frame is given, the last frame will be the end
    length = len(universe.trajectory)
    if args.end==None:
        end = length
        
    # Go through the trajectory
    for ts in universe.trajectory[args.begin:end:args.skip]:
        
        # Timestep properties
        print "Frame %4d" % ts.frame
        t   = ts.time # in ps    
        dim = ts.dimensions[0:3]
        
        # Make a list of centers of mass of all CNTs
        com_list = []
        for tube in cnt_residues:
            com_list.append(tube.center_of_mass())
        com_list = np.array(com_list)
        
        # Calculate the number of clusters and the size of the biggest one
        num, size = clustering(com_list, dim, args.cutoff)
    
        # Write data to file
        output.write("%12.3f \t %8d \t %8d\n" % (t, num, size))


# Calculate the elapsed time
end  = time.time()
elap = end - start
print 'Run time:', elap, 's'

Frame    0
Frame    1
Frame    2
Frame    3
Frame    4
Frame    5
Frame    6
Frame    7
Frame    8
Frame    9
Frame   10
Frame   11
Frame   12
Frame   13
Frame   14
Frame   15
Frame   16
Frame   17
Frame   18
Frame   19
Frame   20
Frame   21
Frame   22
Frame   23
Frame   24
Frame   25
Frame   26
Frame   27
Frame   28
Frame   29
Frame   30
Frame   31
Frame   32
Frame   33
Frame   34
Frame   35
Frame   36
Frame   37
Frame   38
Frame   39
Frame   40
Frame   41
Frame   42
Frame   43
Frame   44
Frame   45
Frame   46
Frame   47
Frame   48
Frame   49
Frame   50
Frame   51
Frame   52
Frame   53
Frame   54
Frame   55
Frame   56
Frame   57
Frame   58
Frame   59
Frame   60
Frame   61
Frame   62
Frame   63
Frame   64
Frame   65
Frame   66
Frame   67
Frame   68
Frame   69
Frame   70
Frame   71
Frame   72
Frame   73
Frame   74
Frame   75
Frame   76
Frame   77
Frame   78
Frame   79
Frame   80
Frame   81
Frame   82
Frame   83
Frame   84
Frame   85
Frame   86
Frame   87
Frame   88
Frame   89
Frame   90